# Pandas groupby + transpose demonstration
The goal is to tranpose a certain columns based on the grouping of other columns

In [1]:
import pandas as pd
import numpy as np
import timeit

In [2]:
# Define parameters
repeat = 10000
df = pd.DataFrame({'device_id': np.ravel([(x, x, x) for x in range(repeat)]), 'run_id': ['run1', 'run2', 'run3'] * repeat})
df_tmp = pd.DataFrame({'device_id':[-1, -1, -1], 'run_id': ['run0', 'run1', 'run2']})
df = pd.concat((df_tmp, df))
df.head()

,device_id,run_id
0,-1,run0
1,-1,run1
2,-1,run2
0,0,run1
1,0,run2


In [3]:
# Option 1: using pivot. This is by far the fastest solution. However, the downside is it might add redundant columns
def func():
    return df.pivot(index='device_id', columns='run_id', values='run_id')

print("Total run time: {0:.2f}s".format(timeit.timeit(func, number=1)))
df_rlt = func()
df_rlt.head()

Total run time: 0.02s


run_id,run0,run1,run2,run3
device_id,,,,
-1,run0,run1,run2,None
0,None,run1,run2,run3
1,None,run1,run2,run3
2,None,run1,run2,run3
3,None,run1,run2,run3


In [4]:
# Option 2: groupby + transpose. This approach is not very fast. Another downside is it creates duplicated indices
def func():
    return df.groupby('device_id')['run_id'].apply(lambda x: x.reset_index(drop=True).to_frame().transpose())

print("Total run time: {0:.2f}s".format(timeit.timeit(func, number=1)))
df_rlt = func()
df_rlt.head()

Total run time: 6.74s


,,0,1,2
device_id,,,,
-1,-1,run0,run1,run2
0,0,run1,run2,run3
1,1,run1,run2,run3
2,2,run1,run2,run3
3,3,run1,run2,run3


In [5]:
# Option 3: loop through groups -- the slowest approach
def func():
    # A workaround solution is to loop through groups
    df_rlt = pd.DataFrame()
    for name, group in df.groupby('device_id'):
        df_rlt = df_rlt.append(group['run_id'].reset_index(drop=True))
    return df_rlt

print("Total run time: {0:.2f}s".format(timeit.timeit(func, number=1)))
df_rlt = func()
df_rlt.head()

Total run time: 20.40s


,0,1,2
run_id,run0,run1,run2
run_id,run1,run2,run3
run_id,run1,run2,run3
run_id,run1,run2,run3
run_id,run1,run2,run3


In [6]:
# Option 4: still using pivot but without adding redundant columns. The trick is to create a rank column and pass it to pivot
def func():
    df_tmp = df
    df_tmp['rank'] = df.groupby('device_id')['run_id'].rank().astype(int)
    return df_tmp.pivot(index='device_id', columns='rank', values='run_id')

print("Total run time: {0:.2f}s".format(timeit.timeit(func, number=1)))
df_rlt = func()
df_rlt.head()

Total run time: 1.33s


rank,1,2,3
device_id,,,
-1,run0,run1,run2
0,run1,run2,run3
1,run1,run2,run3
2,run1,run2,run3
3,run1,run2,run3
